In [2]:
import ee
from datetime import datetime
from dotenv import load_dotenv
import geemap
import geopandas as gpd
import os

In [3]:
EE_PROJECT_ID = os.environ.get("EE_PROJECT_ID")

In [4]:
ee.Authenticate()
ee.Initialize(project=EE_PROJECT_ID)

In [22]:
aoi = gpd.read_file("../data/adm_2/lao_admbnda_adm2_ngd_20191112.shp").explode()
aoi_geometry = aoi[aoi["ADM2_EN"] == "Chomphet"]["geometry"].iloc[0]
coords = [list(aoi_geometry.exterior.coords)]
aoi_gee = ee.Geometry.Polygon(coords)

In [6]:
# Applies scaling factors.
def apply_scale_factors_l5_l7(image):
    optical_bands = image.select("SR_B.").multiply(0.0000275).add(-0.2)
    thermal_bands = image.select("ST_B6").multiply(0.00341802).add(149.0)
    return image.addBands(optical_bands, None, True).addBands(thermal_bands, None, True)


def apply_scale_factors_l8(image):
    optical_bands = image.select("SR_B.").multiply(0.0000275).add(-0.2)
    thermal_bands = image.select("ST_B.*").multiply(0.00341802).add(149.0)
    return image.addBands(optical_bands, None, True).addBands(thermal_bands, None, True)


def add_evi_l5_l7(image):
    # Landsat 5,7: B3(Red), B4(NIR), B1(Blue)
    evi = image.expression(
        "2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))",
        {
            "NIR": image.select("SR_B4"),
            "RED": image.select("SR_B3"),
            "BLUE": image.select("SR_B1"),
        },
    ).rename("EVI")
    return image.addBands(evi)


def add_evi_l8(image):
    # Landsat 8: B4(Red), B5(NIR), B2(Blue)
    evi = image.expression(
        "2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))",
        {
            "NIR": image.select("SR_B5"),
            "RED": image.select("SR_B4"),
            "BLUE": image.select("SR_B2"),
        },
    ).rename("EVI")
    return image.addBands(evi)

In [73]:
def mask_qa(image):
    qa = image.select("QA_PIXEL")
    mask = qa.bitwiseAnd(1 << 3).eq(0)
    return image.updateMask(mask)


# Landsat 5 (1990-2011)
dataset_l5 = (
    ee.ImageCollection("LANDSAT/LT05/C02/T1_L2")
    .filterDate("1990-01-01", "2011-12-31")
    .filterBounds(aoi_gee)
    # .filter(ee.Filter.lt("CLOUD_COVER", 70))
    .map(mask_qa)
    .map(apply_scale_factors_l5_l7)
    .map(add_evi_l5_l7)
)

# Landsat 7 (2012-2013)
dataset_l7 = (
    ee.ImageCollection("LANDSAT/LE07/C02/T1_L2")
    .filterDate("2012-01-01", "2013-12-31")
    .filterBounds(aoi_gee)
    #.filter(ee.Filter.lt("CLOUD_COVER", 70))
    .map(mask_qa)
    .map(apply_scale_factors_l5_l7)
    .map(add_evi_l5_l7)
)

# Landsat 8 (2014-2023)
dataset_l8 = (
    ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")
    .filterDate("2014-01-01", "2023-12-31")
    .filterBounds(aoi_gee)
    #.filter(ee.Filter.lt("CLOUD_COVER", 70))
    .map(mask_qa)
    .map(apply_scale_factors_l8)
    .map(add_evi_l8)
)

In [63]:
def download_image(image, output_dir):
    date = ee.Date(image.get("system:time_start")).format("YYYY-MM-dd").getInfo()
    filename = f"{date}.tif"
    filepath = os.path.join(output_dir, filename)

    image_evi = image.select("EVI")

    geemap.ee_export_image(
        image_evi,
        filepath,
        scale=30,
        region=aoi_gee,
        file_per_band=False,
        crs="EPSG:4326",
    )
    return filename, False

In [ ]:
output_dir = "../data/landsat_images"
os.makedirs(output_dir, exist_ok=True)

In [78]:
all_datasets = dataset_l5.merge(dataset_l7).merge(dataset_l8)
image_list = all_datasets.toList(all_datasets.size())
size = image_list.size().getInfo()

In [79]:
print(size)

673


In [80]:
for i in range(size):
    image = ee.Image(image_list.get(i))
    filename, _ = download_image(image, output_dir)

Generating URL ...
Please wait ...
Data downloaded to /Users/osako/github/aeo-project/python/data/landsat_images/1990-01-15.tif
Generating URL ...
Please wait ...
Data downloaded to /Users/osako/github/aeo-project/python/data/landsat_images/1990-03-04.tif
Generating URL ...
Please wait ...
Data downloaded to /Users/osako/github/aeo-project/python/data/landsat_images/1990-04-21.tif
Generating URL ...
Please wait ...
Data downloaded to /Users/osako/github/aeo-project/python/data/landsat_images/1990-05-07.tif
Generating URL ...
Please wait ...
Data downloaded to /Users/osako/github/aeo-project/python/data/landsat_images/1990-06-08.tif
Generating URL ...
Please wait ...
Data downloaded to /Users/osako/github/aeo-project/python/data/landsat_images/1990-08-27.tif
Generating URL ...
Please wait ...
Data downloaded to /Users/osako/github/aeo-project/python/data/landsat_images/1990-09-12.tif
Generating URL ...
Please wait ...
Data downloaded to /Users/osako/github/aeo-project/python/data/landsa

In [55]:
gfc = ee.Image("UMD/hansen/global_forest_change_2023_v1_11").select(
    ["loss", "lossyear"]
)

geemap.ee_export_image(
    gfc,
    filename="../data/global_forest_change_2023_v1_11.tif",
    scale=30,
    region=aoi_gee,
    crs="EPSG:4326",
    file_per_band=False,
)

Generating URL ...
Please wait ...
Data downloaded to /Users/osako/github/aeo-project/python/data/global_forest_change_2023_v1_11.tif
